In [ ]:
!pip install langchain-google-genai

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB 1.0 MB/s eta 0:00:00
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
   ---------------------------------------- 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.4 MB 5.7 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.4 MB 5.9 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 5.3 MB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 6.0 MB/s eta 0:00:01
   ------------------------------------- -- 1.3/1.4 MB 5.8 MB/s eta 0:00:01
   --------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv()

#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])


In [5]:
print(llm("what is AGI vs Ai, Tell me in short"))

AI is broad; it encompasses any technique that enables computers to mimic human intelligence.  AGI (Artificial General Intelligence) is a *subset* of AI –  it refers to AI with human-level intelligence across *all* tasks, not just specialized ones.  AI is here; AGI is still hypothetical.


#Connect with database and ask some basic questions

In [10]:
!pip install langchain_experimental
!pip install psycopg2-binary



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
#from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
#from langchain_community.utilities.sql_database import SQLDatabase
from langchain.utilities import SQLDatabase

In [15]:


db_user = "postgres"         # Replace with your PostgreSQL username
db_password = "pgsql123" # Replace with your PostgreSQL password
db_host = "localhost"
db_port = "5432"             # Default PostgreSQL port
db_name = "retail_sales_db"

# PostgreSQL connection string using psycopg2
db = SQLDatabase.from_uri(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",
    sample_rows_in_table_info=3  # Fetch 3 sample rows from each table to display in metadata
)

print("Database connection established!")
print(db.table_info)


Database connection established!

CREATE TABLE sales_tb (
	transaction_id INTEGER, 
	transaction_date DATE, 
	customer_id VARCHAR(10), 
	gender VARCHAR(10), 
	age INTEGER, 
	product_category VARCHAR(50), 
	quantity INTEGER, 
	price_per_unit NUMERIC(10, 2), 
	total_amount NUMERIC(10, 2)
)

/*
3 rows from sales_tb table:
transaction_id	transaction_date	customer_id	gender	age	product_category	quantity	price_per_unit	total_amount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [22]:
#Convert question to SQL query
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there"})

# Example: extract just the SQL part from the response
# This works if the response contains 'SQLQuery: ...'
if "SQLQuery:" in response:
    cleaned_query = response.split("SQLQuery:")[-1].strip()
else:
    cleaned_query = response.strip()  # fallback if formatting is different

print("Cleaned query:\n", cleaned_query)
result = db.run(cleaned_query)
print("Result:", result)

# response = chain.invoke({"question": "How many customers are there"})
# response


Cleaned query:
 SELECT COUNT(DISTINCT "customer_id") AS "Number of Customers" FROM sales_tb;
Result: [(29,)]


In [25]:
from sqlalchemy.exc import ProgrammingError

chain = create_sql_query_chain(llm, db)

def execute_query(question):
    try:
        # Generate SQL query from question
        response = chain.invoke({"question": question})
        print(response)
        print("###################################################")

        # Extract SQL query part from response
        if "SQLQuery:" in response:
            cleaned_query = response.split("SQLQuery:")[-1].strip()
        else:
            cleaned_query = response.strip()
        
        print(cleaned_query)
        print("###################################################")        

        # Execute the cleaned query
        result = db.run(cleaned_query)
        print("###################################################")        

        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")


In [26]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

Question: How many unique customers are there for each product category
SQLQuery: SELECT "product_category", COUNT(DISTINCT "customer_id") AS "unique_customer_count" FROM sales_tb GROUP BY "product_category" LIMIT 5;
###################################################
SELECT "product_category", COUNT(DISTINCT "customer_id") AS "unique_customer_count" FROM sales_tb GROUP BY "product_category" LIMIT 5;
###################################################
###################################################
[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [27]:
q2 = "Calculate total sales amount per product category"
execute_query(q2)

Question: Calculate total sales amount per product category
SQLQuery: SELECT "product_category", SUM("total_amount") AS "Total Sales" FROM sales_tb GROUP BY "product_category" LIMIT 5;
###################################################
SELECT "product_category", SUM("total_amount") AS "Total Sales" FROM sales_tb GROUP BY "product_category" LIMIT 5;
###################################################
###################################################
[('Electronics', Decimal('5310.00')), ('Clothing', Decimal('5040.00')), ('Beauty', Decimal('1455.00'))]
